<a href="https://colab.research.google.com/github/alessandronascimento/pyLiBELa/blob/main/Colabs/pyLiBELa_ligand_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quick Instructions

## Choose your runtime:

If you want to use CUDA, a GPU computing API that increases the speed in certain operations, you need to check the ```use_cuda``` box and select the GPU runtime from Google Colab by following the steps in the menu:

> ### `Runtime > Change Runtime Type > T4 GPU`


In case you don't want to use (or don't have acess to) a GPU, pyLiBELa will also work smoothly, but the Grid generation will take longer.

## Required files:

To run pyLiBELa in this notebook, you will need:
* A receptor file in MOL2 format.
* A reference ligand in MOL2 format.
* A ligand to be docked in SMILES.

Make sure that the MOL2 files include hydrogen atoms and atomic charges. You can add hydrogens and charges using the DockPrep tool available in [UCSF Chimera](https://www.cgl.ucsf.edu/chimera/) or UCSF [ChimeraX](https://www.cgl.ucsf.edu/chimerax/)

In [1]:
#@title Installing pyLiBELa (~3 min) {display-mode: "form"}
#@markdown The instalation process requires it to reestart the Colab. Don't be afraid if it happens.

#Installing Dependencies
%%capture
! apt-get install python-dev-is-python3 zlib1g-dev libeigen3-dev libgsl-dev libnlopt-cxx-dev libgsl-dev #libopenbabel-dev openbabel
!pip install ipython-autotime
!pip install py3Dmol
!pip install shutil
!pip install joblib
%load_ext autotime
!pip3 install condacolab
import condacolab
condacolab.install()
!mamba install openbabel

#Compiling pyLiBELa
use_cuda = False #@param {type:"boolean"}

! rm -f Makefile*
! rm -rf obj src test
! rmdir obj src
if use_cuda:
  ! git clone --branch alex-works https://github.com/alessandronascimento/pyLiBELa.git
  ! wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/alex-works/Colabs/Makefile
else:
  ! git clone --branch main https://github.com/alessandronascimento/pyLiBELa.git
  ! wget https://raw.githubusercontent.com/alessandronascimento/pyLiBELa/main/Colabs/Makefile

! mv pyLiBELa/src src
! mv pyLiBELa/test test
! rm -rf pyLiBELa
! mkdir -p obj
! sed -i 's+-I/usr/include/openbabel3+-I/usr/local/include/openbabel3+g' Makefile
! make -j2


time: 3min 11s (started: 2024-04-05 00:46:22 +00:00)


In [1]:
#@title Settings {display-mode: "form"}

import os
import timeit
import numpy as np
import glob
from joblib import Parallel, delayed

try:
  from pyPARSER import *
  from pyMol2 import *
  from pyWRITER import *
  from pyGrid import *
  from pyCOORD_MC import *
  from pyFindHB import *
  from pyEnergy2 import *
  from pyGaussian import *
  from pyConformer import *
  from pyRAND import *
  from pyMcEntropy import *
  from pySA import *
  from pyOptimizer import *
  from pyMC import *
  from pyFullSearch import *
  from pyDocker import *
  print('pyLiBELa is imported!')
except ImportError:
  print('An ImportError occurred, try running this cell again!')


Input = PARSER()
Input.output = "KpTest" #@param {type:"string"}
search_box = 12.0 #@param {type:"number"}
Input.atom_limit = 100 #@param {type:"number"}

Input.generate_conformers = True;
Input.dock_parallel = False;
Input.parallel_jobs = 1;
Input.write_grids = False;
Input.load_grid_from_file = False;
Input.use_grids = False;
Input.write_mol2 = True

Input.dielectric_model = "r"
#Input.diel = 2.0;
Input.scoring_function = 0
Input.grid_prefix = 'McGrid'
Input.solvation_alpha = 0.1
Input.solvation_beta = -0.005

# Optimization parameter:
Input.overlay_optimizer = "mma" #@param ["mma", "ln_auglag", "subplex", "none"]
Input.energy_optimizer = "none"
Input.min_tol = 1E-5 #@param {type:'number'}
Input.min_delta = 1E-5 #@param {type:'number'}
Input.dock_min_tol = 1E-5
Input.timeout = 20 #@param {type:'integer'}
Input.min_timeout = 20

if (Input.scoring_function < 3):
    delta = 2.5
    Input.deltaij6 = (delta*delta*delta*delta*delta*delta)
    delta_es = 2.5
    Input.deltaij_es6 = pow(delta_es, 6);
    Input.deltaij_es3 = (delta_es*delta_es*delta_es)

Input.conf_search_trials = 10000;
Input.conformers_to_evaluate = 2;
Input.lig_conformers = 10;
Input.sort_by_energy = False;

Input.use_score_optimizer = False

Input.search_box_x, Input.search_box_y, Input.search_box_z = search_box, search_box, search_box;

Input.ligand_energy_model = "MMFF94";
Input.atomic_model_ff = "AMBER";

Input.use_writeMol2_score_cutoff = True #@param {type:'boolean'}
Input.writeMol2_score_cutoff = 0.7 #@param {type:'number'}


def mol2_to_pdb(mol2file, pdbfile):
  !obabel -imol2 {mol2file} -opdb -O {pdbfile} > /dev/null

def join_pdb(name_file_rec,name_file_lig,name_file_merge):
  !cp $name_file_rec temp_rec.pdb
  !cp $name_file_lig temp_lig.pdb

  !sed -i '/END\|CONECT/d' temp_rec.pdb #tira o end e o conect do arquivo do receptor
  !sed -i 's/A  /B  /g' temp_lig.pdb #troca a cadeia A por B no arquivo do ligante
  !sed -i '/CONECT/d' temp_lig.pdb #tira o conect do arquivo do ligante
  !cat temp_rec.pdb temp_lig.pdb > $name_file_merge #concatena os arquivos
  !rm -rf temp_rec.pdb temp_lig.pdb

%cd '/content/'
!mkdir -p uploads
!mkdir -p results

Input.output = 'results/' + Input.output
Input.grid_prefix = 'results/' + Input.grid_prefix


pyLiBELa is imported!
/content


In [2]:
#@title Uploading files{display-mode: "form"}
#@markdown Upload your reference ligand file in MOL2 format.
def File_extension_manager(name):
  role_flag = False

  name_ext = name.split('.')
#  name_ext = name_ext[-1]

  if name_ext[-1] == 'mol2':

    role_flag = True
    name_ext_string = '.mol2'

  elif name_ext[-2] == 'mol2':

    role_flag = True
    name_ext_string = '.mol2.gz'

  else:
    print('Check your file extension!\nIt needs to be .mol2 or .mol2.gz.')
    name_ext_string =''

  return role_flag,name_ext_string


import ipywidgets as widgets
from IPython.display import display
from google.colab import files

%cd 'uploads'
!rm -rf *

uploaded = files.upload() #becomes a dictionary, for some reason
up_list = list(uploaded.keys())


rec_toggle = widgets.ToggleButtons(description='Receptor:',
                                   style = {'description_width': 'initial'},
                                   options=up_list)

lig_toggle = widgets.ToggleButtons(description='Reference Ligand:',
                                   style = {'description_width': 'initial'},
                                   options=up_list)

display(rec_toggle)
display(lig_toggle)

def rec_changed(changes):
  rec_name = rec_toggle.value
  #print('receptor em',rec_name)

def lig_changed(changes):
  lig_name = lig_toggle.value
  #print('ligante em',lig_name)

rec_toggle.observe(rec_changed,'value')
lig_toggle.observe(lig_changed,'value')
#print(toggle.value)

/content/uploads


Saving KpThiL_lig.mol2.gz to KpThiL_lig.mol2.gz


ToggleButtons(description='Receptor:', options=('KpThiL_lig.mol2.gz',), style=ToggleButtonsStyle(description_w…

ToggleButtons(description='Reference Ligand:', options=('KpThiL_lig.mol2.gz',), style=ToggleButtonsStyle(descr…

In [3]:
#@title Reading search ligands {display-mode: "form"}
#@markdown If you want to use your own dataset, just upload a file with .smi extensions into the **uploads** folder. The file must have one compound per line with a single-word description.
#@markdown Example: OC(=O)CC1=CNC2=C1C=CC=C2 MOL000000.

%cd /content

# Getting smiles from ZINC
use_chembl30 = False # @param {type:"boolean"}
use_chembi = False # @param {type:"boolean"}
use_drugbank = False # @param {type:"boolean"}
use_fda = True # @param {type:"boolean"}
use_hmdbdrug = False # @param {type:"boolean"}
chunck_size = 100 # @param {type: "integer"}

def read_multimol(multimol):
  multifile = open(multimol, 'r')
  molecules_to_dock  = []
  mol_names = []
  for line in multifile:
    line2 = line.split()
    molecules_to_dock.append(line2[0])
    mol_names.append(line2[1])
  multifile.close()
  return molecules_to_dock, mol_names

def dock_mol(mol, Dock, Rec, RefLig, center, Input, molname, count):
  Lig2 = Mol2();
  try:
    Lig2.parse_smiles(Input, mol, molname);
    try:
      Dock.run(Rec, Lig2, RefLig, center, Input, count);
    except:
      print("Could not dock molecule %s" % molname);
  except:
    print("Could not parse molecule %s" % molname);

if (use_chembl30):
  !curl -O https://files.docking.org/catalogs/source/chembl30.src.txt
  ! mv chembl30.src.txt uploads/chembl30.smi
if (use_chembi):
  !curl -O https://files.docking.org/catalogs/source/chebi.src.txt
  !mv chebi.src.txt uploads/chebi.smi
if (use_drugbank):
  !curl -O https://files.docking.org/catalogs/source/dbap.src.txt
  !mv dbap.src.txt uploads/dbpap.smi
if (use_fda):
  !curl -O https://files.docking.org/catalogs/source/dbfda.src.txt
  !mv dbfda.src.txt uploads/dbfda.smi
if (use_hmdbdrug):
  !curl -O https://files.docking.org/catalogs/source/hmdbdrug.src.txt
  !mv hmdbdrug.src.txt uploads/hmdbdrug.smi

!rm -f uploads/multimol.smi
!cat uploads/*.smi > uploads/multimol.smi
! grep -v smiles uploads/multimol.smi > tmp
! mv tmp uploads/multimol.smi

!split -l {chunck_size} -d uploads/multimol.smi uploads/multimol.smi.
chuncks = glob.glob('uploads/multimol.smi.*')
print()
print()
print('Input file divided into %d chuncks with %d molecules.' % (len(chuncks), chunck_size))
print()



/content
curl: /usr/local/lib/libcurl.so.4: no version information available (required by curl)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  102k  100  102k    0     0   780k      0 --:--:-- --:--:-- --:--:--  785k


Input file divided into 14 chuncks with 100 molecules.



In [ ]:
#@title Similarity Search (might take a while)
# Remove previous results, if any
! rm -f {Input.output}_dock.mol2.gz lig2.pdb

Rec = Mol2(Input, 'uploads/' + lig_toggle.value)
RefLig = Mol2(Input, 'uploads/' + lig_toggle.value)
Coord = COORD_MC()
center = Coord.compute_com(RefLig)
Writer = WRITER(Input)
Dock = Docker(Writer)

print()
print('Starting docking calculation...')
print()
for chunck in chuncks:
  print('Docking chunck %s' % chunck)
  molecules_to_dock, mol_names = read_multimol(chunck);
  print('Going parallel...')
  Parallel(backend='threading', n_jobs=1)(delayed(dock_mol)(molecules_to_dock[i], Dock, Rec, RefLig, center, Input, mol_names[i], i+1) for i in range(len(molecules_to_dock)))
  print()
print()
print('Docking calculation finished!')
print()


Starting docking calculation...

Docking chunck uploads/multimol.smi.11
Going parallel...


In [ ]:
#@title Downloading files
import shutil
jobname = Input.output.split('/')
results = 'pyLiBELa_{}'.format(jobname[1])
print('File {} is being prepared'.format(results))
shutil.make_archive(results, 'zip', results)
files.download('{}'.format(results))